In [5]:
import pandas as pd

In [6]:
pdf_df = pd.read_excel("pdf_links.xlsx")
pdf_df.head()

,pdf_link
0,https://www.cncbinternational.com/_document/ab...
1,https://www.cncbinternational.com/_document/im...
2,https://www.cncbinternational.com/_document/ab...
3,https://www.cncbinternational.com/_document/ac...
4,https://www.cncbinternational.com/_document/fo...


In [7]:
traversed_df = pd.read_excel("traversed_links.xlsx")
traversed_df

,traversed_link
0,https://www.cncbinternational.com/home/en/inde...
1,https://www.cncbinternational.com/personal/inv...
2,https://www.cncbinternational.com/internationa...
3,https://www.cncbinternational.com/personal/e-b...
4,https://www.cncbinternational.com/personal/mor...
...,...
331,https://www.cncbinternational.com/buy_order.html
332,https://www.cncbinternational.com/index
333,https://www.cncbinternational.com/cash-in-inst...
334,https://www.cncbinternational.com/airport_limo...


In [ ]:
traversed_df[traversed_df["traversed_link"].str.contains("monopoly")]